# Projeto Final - PFSP TT - Dupla 09

**Integrantes:**

* Lucas Hideki Takeuchi Okamura NUSP: 9274315

* Thales Arantes Kerche Nunes NUSP: 10769372

**Objetivo**

* Programar o algoritmo heurístico NEH, que busca minimizar o makespan em um flow shop com m máquinas

**Descrição**

O código roda o algoritmo de NEH, outro método para resolver o problema
de máquinas em série. O algoritmo está explicitado no artigo "A Heuristic 
Algorithm for the m-Machine, n-Job Flow-shop Sequencing Problem", 1983, por 
Nawaz, Muhammad; Jr, E Emory Enscore; Ham, Inyong. Resumidamente, tem-se 
os passos abaixo:

	* Passo 1: Ordenar a lista de jobs em ordem decrescente do tempo total de processamento;

	* Passo 2: Retire os dois primeiros da lista ordenada e crie uma nova lista, teste as duas permutações, escolha a
    permutação com menor makespan;

	* Passo 3: Retire o próximo job da lista ordenada, teste em todas as posições da nova lista, escolha a inserção
    com menor makespan;

	* Passo 4: Repita o passo 3 até esvaziar a lista ordenada; a nova lista resultante será a solução do problema

In [27]:
import xlwings as xw
import numpy as np
import random
import math
import time

In [2]:
class Job:
    def __init__(self, i: int, pi: list, di: int):
        self.i=i     # número do job, pela ordem de chegada
        self.p=pi     # processing time
        self.p_total=sum(self.p)
        self.d=di     # due date
        self.C=0     # completion time
        self.T=0     # tardiness

In [3]:
def leInst(sheet_num: int) -> (int, int, list):
    pl1 = wb1.sheets[sheet_num]
    m=int(pl1.range('B5').value)     # converter em inteiro
    n=int(pl1.range('B6').value)     # converter em inteiro
    tab1=pl1.range('B11').options(expand='table', numbers=int).value     # ler como tabela e inteiros
    print()
    print('n =',n)
    print('m =',m)
    print('dados =',tab1)
    jobs=[]
    for i in range(n):
        pi=tab1[i][:-1]
        di=tab1[i][-1]
        jobs.append(Job(i,pi,di))

    return m,n,jobs

In [4]:
def gravaSched(i: int,jobs: list,dados: dict):
    n=len(jobs)
    m=len(jobs[0].p)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C+[job.d]+[job.T]
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)

    plan.range('A5').value=['m',m]
    plan.range('A6').value=['n',n]
    plan.range('E2').value=['Cmax',dados['Cmax']]
    plan.range('E3').value=['Fbar',dados['Fbar']]
    plan.range('E4').value=['TT',dados['TT']]
    plan.range('E5').value=['Tmax',dados['Tmax']]
    plan.range('E6').value=['nT',dados['nT']]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5', 'd', 'T']
    plan.range('A11').value=tab

In [5]:
def calcCmax(m: int,jobs: list) -> int:     # esta função calcula os tempos de conclusão dos jobs nas m máquinas
    n=len(jobs)
    c=[[0 for i in range(m)] for j in range(n)]
    p=[job.p for job in jobs]

    # primeiro job
    c[0][0]=p[0][0]     # primeira máquina
    for k in range(1,m):     # próximas máquinas
        c[0][k]=c[0][k-1]+p[0][k]

    # próximos jobs
    for j in range(1,n):
        c[j][0]=c[j-1][0]+p[j][0]     # primeira máquina
        for k in range(1,m):     # próximas máquinas
            c[j][k]=max(c[j][k-1],c[j-1][k])+p[j][k]

    j=0
    for job in jobs:
        job.C=c[j]
        j+=1

    return c[-1][-1]

def calculate_final_values(jobs: list, m: int = 5) -> (dict, list):

    Cmax = calcCmax(m, jobs)
    TT = nT = Fbar = Tmax = 0
    for job in jobs:
        job.T = max(0, job.C[-1] - job.d)
        TT += job.T
        Fbar += job.C[-1]
        if job.T > 0:
            nT += 1
            Tmax = max(Tmax, job.T)

    Fbar = Fbar/len(jobs)

    return {'Cmax': Cmax, 'Fbar': Fbar, 'TT': TT, 'Tmax': Tmax, 'nT': nT}, jobs

def findBestPerm(jobs: list, job_insert: Job, m: int = 5) -> (dict, list):

    minTardiness = np.inf

    for j in range(len(jobs) + 1):
        totTardiness = 0
        jobs.insert(j, job_insert)
        dados, jobs = calculate_final_values(jobs)
        if dados['TT'] < minTardiness:
            bestPos = j
            minTardiness = dados['TT']
        jobs.pop(j)
    jobs.insert(bestPos, job_insert)

    dados, jobs = calculate_final_values(jobs)

    return dados, jobs

In [6]:
def schedNEHedd(jobs: list, m: int = 5) -> (list, int):
    n = len(jobs)
    jobs.sort(key = lambda job: job.d, reverse = False)
    jobs1 = [jobs[0]]

    for k in range(1, n):
        dados, jobs1 = findBestPerm(jobs1, jobs[k])

    dados, _ = calculate_final_values(jobs1)

    return jobs1, dados

In [19]:
def schedLS(jobs: list, bestTT: int, m: int = 5) -> (list, int):

    melhoria = True
    aux_jobs = jobs[:]
    while melhoria:
        melhoria = False
        jotas = list(range(len(aux_jobs)))
        random.shuffle(jotas)

        for j1 in jotas:
            job1 = aux_jobs.pop(j1)
            bestPos = j1

            for j2 in range(len(aux_jobs)+1):
                aux_jobs.insert(j2, job1)
                dados, aux_jobs = calculate_final_values(aux_jobs)

                if dados['TT'] < bestTT:
                    bestPos = j2
                    melhoria = True
                    bestTT = dados['TT']
                    break
                aux_jobs.pop(j2)
            if melhoria:
                break
            aux_jobs.insert(bestPos, job1)

    dados, aux_jobs = calculate_final_values(aux_jobs)

    return aux_jobs, dados

In [22]:
def Temperatura(T: float, jobs: list, n: int = 20, m: int = 5) -> float:
    return (T*sum([job.p_total for job in jobs])/(10*n*m))

def DestructionConstruction(jobs_linha: list, d: int, m: int):
    
    retirados = random.sample(jobs_linha, d)
    for item in retirados:
        jobs_linha.remove(item)
    for new_job in retirados:
        dados, jobs_linha = findBestPerm(jobs_linha, new_job, m)
        
    return jobs_linha

def schedIG(jobs: list, T: float = 0.5, d: int = 4, m: int = 5) -> (list, dict):

    jobs_b = jobs[:]
    for _ in range(100):

        jobs_linha = DestructionConstruction(jobs[:], d, m)
        jobs_linha, dados = schedLS(jobs_linha, calculate_final_values(jobs_linha)[0]['TT'], m)

        if dados['TT'] < calculate_final_values(jobs)[0]['TT']:
            jobs = jobs_linha[:]
            if dados['TT'] < calculate_final_values(jobs_b)[0]['TT']:
                jobs_b = jobs_linha[:]
        elif random.random() <= math.exp(-(dados['TT']-calculate_final_values(jobs)[0]['TT'])/Temperatura(T, jobs)):
            jobs = jobs_linha[:]

    dados, jobs = calculate_final_values(jobs)

    return jobs, dados

# Testes

In [29]:
wb1 = xw.Book('xl15 2 A PFSP TT.xlsx')
# wb2 = xw.Book()
I = wb1.sheets.count     # número de instância é igual ao número de planilhas na pasta
d = 4
T = 1
time_list = []

for i in range(I):
    print()
    print(i+1)
    m, n, jobs_i = leInst(i)
    
    start_time = time.time()

    jobs, dados = schedNEHedd(jobs_i[:], m)

    jobs, dados1 = schedLS(jobs, dados['TT'], m)

    best_jobs, best_dados = schedIG(jobs, T, d, m)
    
    time_list.append(time.time()-start_time)

    print(best_dados)
    print(time_list[i])

#     gravaSched(i, best_jobs, best_dados)


1

n = 20
m = 5
dados = [[58, 60, 88, 14, 46, 816], [34, 33, 17, 67, 15, 613], [54, 25, 93, 60, 2, 929], [19, 84, 25, 47, 8, 895], [24, 45, 25, 16, 11, 1067], [21, 69, 93, 22, 71, 967], [64, 43, 48, 83, 39, 1233], [37, 43, 18, 46, 13, 723], [26, 57, 34, 5, 18, 821], [82, 14, 98, 27, 14, 1116], [84, 43, 96, 67, 81, 915], [75, 87, 45, 11, 44, 1229], [73, 27, 32, 9, 79, 703], [24, 94, 39, 23, 95, 813], [49, 54, 8, 70, 97, 782], [27, 61, 55, 87, 35, 853], [98, 8, 90, 15, 55, 882], [51, 93, 80, 86, 35, 1286], [83, 87, 25, 94, 80, 843], [12, 80, 12, 49, 38, 707]]
{'Cmax': 1329, 'Fbar': 781.45, 'TT': 255, 'Tmax': 93, 'nT': 5}
85.03167724609375

2

n = 20
m = 5
dados = [[96, 12, 54, 58, 95, 805], [68, 46, 39, 53, 90, 1028], [26, 87, 24, 1, 13, 937], [23, 50, 2, 24, 34, 672], [66, 63, 16, 90, 9, 917], [83, 38, 41, 21, 34, 868], [31, 13, 17, 55, 71, 750], [63, 76, 77, 70, 95, 1069], [19, 80, 86, 46, 54, 903], [44, 78, 3, 94, 66, 760], [89, 58, 17, 27, 28, 994], [49, 78, 63, 52, 28, 1217], [32, 

In [9]:
import gurobipy as grb

In [14]:
def schedGurobi(jobs: list, m: int = 5) -> (list, int, float, float):
    
    n = len(jobs)
    
    #Cria o modelo para a intância que será rodada
    mPFSP = grb.Model(name="Permutation Flow Shop Problem")
    x = mPFSP.addVars(n, n, vtype=grb.GRB.BINARY, name='x')
    s = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='s')
    c = mPFSP.addVars(n, m, vtype=grb.GRB.CONTINUOUS, lb=0, name='c')
    T = mPFSP.addVars(n, vtype=grb.GRB.CONTINUOUS, lb=0, name='T')
    TT = mPFSP.addVar(vtype=grb.GRB.CONTINUOUS, lb=0, name='TT')
    mPFSP.update()

    #coloca as constraints necessárias para a otimização
    mPFSP.addConstrs((s[j,k] - c[j,k] + grb.quicksum(jobs[i].p[k]*x[i,j] for i in range(n)) <= 0 for k in range(m) for j in range(n)), name='r1')
    mPFSP.addConstrs((s[j,k]-c[j-1,k]>=0 for k in range(m) for j in range(1, n)), name='r2')
    mPFSP.addConstrs((s[j,k]-c[j,k-1]>=0 for k in range(1, m) for j in range(n)), name='r3')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for i in range(n)) == 1  for j in range(n)), name='r4')
    mPFSP.addConstrs((grb.quicksum(x[i,j] for j in range(n)) == 1  for i in range(n)), name='r5')
    mPFSP.addConstrs((T[j] - c[j,m-1] + grb.quicksum(jobs[i].d*x[i,j] for i in range(n)) >= 0 for j in range(n)), name='r6')
    mPFSP.addConstr(TT >= grb.quicksum(T[i] for i in range(n)), name='r7')
    mPFSP.update()
    
    #define o objetivo
    mPFSP.ModelSense = grb.GRB.MINIMIZE
    mPFSP.setObjective(TT)
    mPFSP.update()
    
    #otimiza o modelo e recupera as informações relevantes
    mPFSP.optimize()
    runtime = mPFSP.Runtime
    TT = round(mPFSP.objVal)
    gap = mPFSP.MIPGap
    
    #pega a posição ideal de cada job do modelo, e 
    pos = np.reshape(mPFSP.getVars()[:n*n], (n, n))
    for i in range(n):
        for j in range(n):
            if pos[i, j].x > 0.999:
                jobs[i].bestPos = j
                break

    jobs.sort(key=lambda job: job.bestPos)
    print([job.i+1 for job in jobs])
    _ = calcCmax(m, jobs)
    
    return jobs, TT, runtime, gap

In [15]:
def gravaSchedGurobi(i: int,jobs: list,TT: int, m: int, Runtime: float, Gap: float):
    n=len(jobs)
    tab=[]
    for job in jobs:
        a=[job.i+1]+job.p+job.C+[job.d]
        tab.append(a)

    if i==0:
        plan=wb2.sheets[0]
        plan.name='I(1)'
    else:
        plan=wb2.sheets.add('I('+str(i+1)+')',after=i)
        
    plan.range('A5').value=['m',m]
    plan.range('A6').value=['n',n]
    plan.range('E5').value=['TT',TT]
    plan.range('H5').value=['Runtime',Runtime]
    plan.range('H6').value=['Gap',Gap]
    plan.range('A10').value=['#','p1','p2', 'p3', 'p4', 'p5','C1','C2','C3','C4','C5','d']
    plan.range('A11').value=tab

    print()
    print('TT (I('+str(i+1)+')): ', TT)
    print()
    print()

In [12]:
wb1 = xw.Book("xl15 2 A PFSP TT.xlsx")
wb2 = xw.Book()
I = wb1.sheets.count

In [16]:
for i in range(I):
    print()
    print(i+1)
    m,n,jobs=leInst(i)

    jobs, TT, runtime, gap = schedGurobi(jobs)

    gravaSchedGurobi(i, jobs, TT, m, runtime, gap)


1

n = 20
m = 5
dados = [[58, 60, 88, 14, 46, 816], [34, 33, 17, 67, 15, 613], [54, 25, 93, 60, 2, 929], [19, 84, 25, 47, 8, 895], [24, 45, 25, 16, 11, 1067], [21, 69, 93, 22, 71, 967], [64, 43, 48, 83, 39, 1233], [37, 43, 18, 46, 13, 723], [26, 57, 34, 5, 18, 821], [82, 14, 98, 27, 14, 1116], [84, 43, 96, 67, 81, 915], [75, 87, 45, 11, 44, 1229], [73, 27, 32, 9, 79, 703], [24, 94, 39, 23, 95, 813], [49, 54, 8, 70, 97, 782], [27, 61, 55, 87, 35, 853], [98, 8, 90, 15, 55, 882], [51, 93, 80, 86, 35, 1286], [83, 87, 25, 94, 80, 843], [12, 80, 12, 49, 38, 707]]
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 336 rows, 621 columns and 3811 nonzeros
Model fingerprint: 0xd8825c85
Variable types: 221 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 29 rows and 30 columns
Presolve time: 0.05s
Presolved: